#### 

# Evaluation Metrics.

### Evaluation Metrics are used to evaluate our models by calculating its accuracy or its loss in different ways, I'll go over a few of them

### 1- Mean absolute error (MAE) -> This is a loss function 
#### Calculated by ∑ |output - predicted value| / samples_number

### 2- Mean absolute percentage error (MAPE) -> This is a loss function 
#### Calculated by ∑ |100*(output - predicted value)/output| / samples_number

### 3- Mean squared error (MSE)  -> This is a loss function
#### Calculated by ∑ (output - predicted value)^2 / samples_number

### 4- Root mean squared error (RMSE)  -> This is a loss function
#### Calculated by ∑ sqrt((output - predicted value)^2) / samples_number

### 5- Root mean squared logarithmic error (RMSLE) -> This is a loss function
#### Calculated by ∑ log(sqrt((output - predicted value)^2)) / samples_number

### 6- Coefficient of determination (R2) -> This is an accuracy function
### 
### Others functions ...
#### 7- Symmetric mean absolute percentage error (sMAPE)
#### 8- Mean absolute scaled error (MASE) 
#### 9- Mean squared prediction error (MSPE)
#### 10- Mean directional accuracy (MDA) 
#### 11- Median absolute deviation (MAD)

#### I have not found functions from 7:11 in sklearn so i'm not sure if they are built-in or can be calculated manually


##### ref : https://www.analyticsvidhya.com/blog/2021/05/know-the-best-evaluation-metrics-for-your-regression-model/
##### ref : https://en.wikipedia.org/wiki/Mean_absolute_percentage_error


### Import libraries

In [2]:
import numpy as np
import pandas as pd

from pandas import concat
from pandas import DataFrame

from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras import callbacks
from tensorflow.keras.callbacks import * 
from tensorflow.keras.models import Model , Sequential
from tensorflow.keras.layers import Input,Conv2D,Conv3D ,TimeDistributed

from numpy import array
from numpy import split
from pandas import read_csv
from keras.layers import LSTM ,Dense,Flatten , TimeDistributed
from keras.layers import RepeatVector

from math import sqrt
from math import log
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error

### Data preprocessing

In [3]:
df = pd.read_csv('household_power_consumption  new  .csv',parse_dates=['datetime'], index_col= 'datetime')

In [4]:
df.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,sub_metering_4
datetime,,,,,,,,
2006-12-17 00:00:00,1.044,0.152,242.73,4.4,0.0,2.0,0.0,15.400000
2006-12-17 00:01:00,1.520,0.220,242.20,7.4,0.0,1.0,0.0,24.333334
2006-12-17 00:02:00,3.038,0.194,240.14,12.6,0.0,2.0,0.0,48.633335
2006-12-17 00:03:00,2.974,0.194,239.97,12.4,0.0,1.0,0.0,48.566666
2006-12-17 00:04:00,2.846,0.198,240.39,11.8,0.0,2.0,0.0,45.433334


### data preparation

In [5]:
def train_test_split(df):
    
    # compute split point
    end_idx = df.shape[0]* 70 // 100
    
    train_data = df.iloc[:end_idx, : ]
    test_data = df.iloc[end_idx:, :]
    
    return train_data, test_data

train, test = train_test_split(df.iloc[:1000,:])

In [6]:
def scale_data(train, test):
    scaler = MinMaxScaler().fit(train)
    return scaler.transform(train), scaler.transform(test), scaler

train_scaled , test_scaled , scaler = scale_data(train, test)

In [7]:
def convert_to_supervised(df):

    input_features = []
    ouput_feature = []
    
    len_df = df.shape[0]
    
    for i in range(len_df):
        
        end_idx = i + 1 
        
        if end_idx > len_df-1:
            break
            
        input_x , output_y = df[i:end_idx, 1:], df[end_idx: end_idx+1, 0]
        
        input_features.append(input_x)
        ouput_feature.append(output_y)
    
    return np.array(input_features), np.mean(np.array(ouput_feature), axis=1)

In [8]:
X_train, y_train = convert_to_supervised(train_scaled)
print(f"X_train shape -> {X_train.shape}\ny_train shape -> {y_train.shape}")

X_train shape -> (699, 1, 7)
y_train shape -> (699,)


In [9]:
X_test, y_test = convert_to_supervised(test_scaled)
print(f"X_test shape -> {X_test.shape}\ny_test shape -> {y_test.shape}")

X_test shape -> (299, 1, 7)
y_test shape -> (299,)


## Time Distributed

In [10]:
def build_model(train , epochs = 500 , batch_size = 100 , verbose = 0):
    
    n_timesteps = X_train.shape[1] 
    n_features = X_train.shape[2]
    n_outputs = 1
        
    model = Sequential()
    model.add(TimeDistributed(Dense(200,input_shape=(n_timesteps, n_features), activation='relu')))
    model.add(TimeDistributed(Dense(1)))
    
    model.compile(loss='mean_squared_error', optimizer='adam')
    
    callback = callbacks.EarlyStopping(monitor='loss', patience=5)
    
    info = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose,shuffle=False , callbacks = callback)
    return model , info
    
y_train = y_train.reshape((y_train.shape[0], 1, 1))
model , info = build_model(X_train)
y_pred = model.predict(X_test, batch_size = 200, verbose = 0)

print(info.params,"\n")
print("Mean absolute error -----------> ", mean_absolute_error( y_test.flatten() , y_pred.flatten() ))
print("Mean squared error ------------> ", mean_squared_error( y_test.flatten() , y_pred.flatten() ))
print("Root mean squared error -------> ", sqrt(mean_squared_error( y_test.flatten() , y_pred.flatten()) ))
print("Root mean squared log error ---> ", log(sqrt(mean_squared_error( y_test.flatten() , y_pred.flatten()) )),sep="")

{'verbose': 0, 'epochs': 500, 'steps': 7} 

Mean absolute error ----------->  0.07836752525399295
Mean squared error ------------>  0.00855826682064903
Root mean squared error ------->  0.09251090109089323
Root mean squared log error ---> -2.3804287917762017


#### 

In [ ]:
"""
class TimeDistributed(Model):
    def __init__(self):
        super(TimeDistributed,self).__init__()
        
        self.timedistributed1 = TimeDistributed()
        self.timedistributed2 = TimeDistributed()
    
        self.dence1 = Dense(100, activation='relu')
        self.dence2 = Dense(1)
    
    def call(self,inputs):
        x = self.timedistributed1(inputs,dence1)
        x = self.timedistributed2(x)
        x = x(sef.dence2)
        return x
    

input_layer = Input(shape= (n_timesteps, n_features))
x = TimeDistributed()(input_layer)
model = Model(inputs = input_layer , outputs= x)


# Kenral dead!

"""